<a href="https://colab.research.google.com/github/stbalaji/AIForBusinessDLNLP/blob/main/AIForBusiness_BERT_Detailed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow.keras.backend as K
import tensorflow as tf

In [8]:
class BertLayer(tf.keras.layers.Layer):

    def __init__(self,  n_fine_tune_layers=12,bert_path= "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", **kwargs):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        self.bert_path = bert_path

        super(BertLayer, self).__init__(**kwargs)


    def build(self, input_shape):
        self.bert = hub.Module(self.bert_path,trainable=self.trainable,name=f"{self.name}_module"
        )
        trainable_vars = self.bert.variables
        trainable_vars =  [var for var in trainable_vars if not "/cls/" in var.name]
        trainable_vars = trainable_vars[-self.n_fine_tune_layers :]

        for var in trainable_vars:
            self._trainable_weights.append(var)

        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]

        input_ids, input_mask, segment_ids = inputs

        bert_inputs = dict(input_ids=input_ids, input_mask=input_mask,segment_ids=segment_ids)
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)["sequence_output"]

        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)


In [9]:
def buildBertModel():
  max_seq_length=256
  in_id = tf.keras.layers.Input(shape=(max_seq_length,),name="input_ids")
  in_mask = tf.keras.layers.Input(shape=(max_seq_length,),name="input_masks")
  in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")

  bert_inputs = [in_id, in_mask, in_segment]

  bert_output = BertLayer(n_fine_tune_layers=10)(bert_inputs)

  drop = keras.layers.Dropout(0.3)(bert_output)
  dense = keras.layers.Dense(200, activation='relu')(bert_output)
  drop = keras.layers.Dropout(0.3)(dense)
  dense = keras.layers.Dense(100, activation='relu')(dense)
  pred = keras.layers.Dense(1, activation='sigmoid')(dense)

  session = K.get_session()
  init = tf.compat.v1.global_variables_initializer()
  session.run(init)

  model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
  model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
  model.summary()
  return model


In [10]:
def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in examples:
      input_id, input_mask, segment_id, label = convert_single_example(
      tokenizer, example, max_seq_length)
      input_ids.append(input_id)
      input_masks.append(input_mask)
      segment_ids.append(segment_id)
      labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1)
    )

In [11]:
import tensorflow.keras.backend as K

def createBertFunction():
  max_seq_length=256
  in_id = tf.keras.layers.Input(shape=(max_seq_length,),name="input_ids")
  in_mask = tf.keras.layers.Input(shape=(max_seq_length,),name="input_masks")
  in_segment = tf.keras.layers.Input(shape=(max_seq_length,),name="segment_ids")

  bert_input = [in_id, in_mask, in_segment]

  bert_output = BertLayer(n_fine_tune_layers=10)(bert_input)

  func = K.function([bert_input], [bert_output])

  session = K.get_session()
  init = tf.compat.v1.global_variables_initializer()
  session.run(init)

  return func


In [13]:
def finetuneBertModel(trainT, valT):
    model=buildBertModel()
    (train_input_ids, train_input_masks, train_segment_ids,train_labels)=trainT
    (val_input_ids,val_input_masks,val_segment_ids,val_labels)=valT

    model.fit(
        [train_input_ids, train_input_masks, train_segment_ids],
        train_labels,
        epochs=10,
        batch_size=64
    )


In [ ]:
def convert_single_example(tokenizer, example, max_seq_length=256):
    tokens_a = tokenizer.tokenize(example.text)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_mask = [1] * len(input_ids)

    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    return (input_ids, input_mask, segment_ids, example.label)
